In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import xgboost as xgb
import lightgbm as lgbm
import catboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import optuna
from optuna.samplers import TPESampler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Loading Data

In [2]:
BASE_DIR = Path("/kaggle/input/playground-series-s3e2/")

train = pd.read_csv(BASE_DIR / "train.csv").drop(columns="id")
test = pd.read_csv(BASE_DIR / "test.csv").drop(columns="id")

# Preprocessing

In [3]:
df = pd.concat([train.drop(columns=["stroke"]), test], axis=0).reset_index(drop=True)
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked


# Feature Engineering

In [4]:
def diabetes_indicator(avg_glucose_level):
    if avg_glucose_level <= 99:
        return "normal"
    elif avg_glucose_level >= 100 and avg_glucose_level <= 125:
        return "prediabetic"
    elif avg_glucose_level <= 200:
        return "type1"
    else:
        return "type2"

df.avg_glucose_level.map(diabetes_indicator).value_counts()

normal         20098
prediabetic     4002
type1            916
type2            492
Name: avg_glucose_level, dtype: int64

In [5]:
df["diabetes"] = df.avg_glucose_level.map(diabetes_indicator)
df.head(3)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,diabetes
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,normal
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,normal
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,prediabetic


# One hot encoding

In [6]:
df = pd.get_dummies(df)
df.head(3)

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,...,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,diabetes_normal,diabetes_prediabetic,diabetes_type1,diabetes_type2
0,28.0,0,0,79.53,31.1,0,1,0,0,1,...,0,1,0,0,1,0,1,0,0,0
1,33.0,0,0,78.44,23.9,0,1,0,0,1,...,1,0,0,1,0,0,1,0,0,0
2,42.0,0,0,103.00,40.3,1,0,0,0,1,...,1,0,1,0,0,0,0,1,0,0


# Preprocessing v2

In [7]:
X = df.iloc[:-len(test), :]
test_new = df.iloc[-len(test):, :]
y = train.stroke

In [8]:
lgbm_params = {'n_estimators': 1449,
                 'num_rounds': 150,
                 'learning_rate': 0.2487851433195791,
                 'num_leaves': 229,
                 'max_depth': 4,
                 'min_data_in_leaf': 903,
                 'lambda_l1': 1.6076796916985906e-05,
                 'lambda_l2': 0.0006193243147464635,
                 'min_gain_to_split': 5.754065628334817,
                 'bagging_fraction': 0.8222441421542225,
                 'feature_fraction': 0.9652642961816826}

In [10]:
lgbm_model = lgbm.LGBMClassifier(objective="binary", is_unbalance=True, **lgbm_params)

In [12]:
lgbm_model.fit(X, y)

[LightGBM] [Warning] lambda_l1 is set=1.6076796916985906e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.6076796916985906e-05
[LightGBM] [Warning] bagging_fraction is set=0.8222441421542225, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8222441421542225
[LightGBM] [Warning] num_iterations is set=150, num_rounds=150 will be ignored. Current value: num_iterations=150
[LightGBM] [Warning] min_gain_to_split is set=5.754065628334817, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=5.754065628334817
[LightGBM] [Warning] lambda_l2 is set=0.0006193243147464635, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0006193243147464635
[LightGBM] [Warning] feature_fraction is set=0.9652642961816826, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9652642961816826
[LightGBM] [Warning] min_data_in_leaf is set=903, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=903


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


LGBMClassifier(bagging_fraction=0.8222441421542225,
               feature_fraction=0.9652642961816826, is_unbalance=True,
               lambda_l1=1.6076796916985906e-05,
               lambda_l2=0.0006193243147464635,
               learning_rate=0.2487851433195791, max_depth=4,
               min_data_in_leaf=903, min_gain_to_split=5.754065628334817,
               n_estimators=1449, num_leaves=229, num_rounds=150,
               objective='binary')

In [14]:
y_pred = lgbm_model.predict_proba(test_new)[:, 1]
y_pred

array([0.44027972, 0.8138018 , 0.00889264, ..., 0.0134876 , 0.02604852,
       0.00958554])

In [15]:
# since we dropped id column from our test dataframe earlier but we need it for submission file so we're reloading it
test_idx = pd.read_csv(BASE_DIR / "test.csv").id
test_idx

0        15304
1        15305
2        15306
3        15307
4        15308
         ...  
10199    25503
10200    25504
10201    25505
10202    25506
10203    25507
Name: id, Length: 10204, dtype: int64

In [16]:
submission = pd.DataFrame(data={"id": test_idx, "stroke": y_pred})
submission.head()

,id,stroke
0,15304,0.440280
1,15305,0.813802
2,15306,0.008893
3,15307,0.606344
4,15308,0.065422


In [17]:
submission.to_csv("submission.csv", index=False)